# COURSERA CAPSTONE PROJECT

### This notebook will be used primarily for the capstone project

In [3]:
# Importing packages
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


# BATTLE OF THE NEIGHBORHOODS - OAK PARK VS. EVANSTON, IL

### INTRODUCTION & BUSINESS PROBLEM

In this notebook we will analyze two afluent suburbs of Chicago, IL.

The suburbs are Evanston and Oak Park. 

Evanston, which is 12 miles (19 km) north of Chicago, is known for being home to Northwestern University, a well-known college.<sup>1</sup>

Oak Park, on the other hand, sits 10 miles (16 km) west of Chicago, and it is known for architect Frank Lloyd Wright and writer Ernest Hemingway. <sup>2</sup>

Both places are the most common consideration for couples with kids looking to move out of Chicago for more space while retaining easy access to the city. 

Evanston and Oak Park have good schools, and similar real estate markets. Access to Chicago is easy by using both train or the subway system, in addition to interstates for car access. 

This project aims to help couples make an informed decision based on their personal preferences. 

<sup>1</sup> https://en.wikipedia.org/wiki/Evanston,_Illinois

<sup>2</sup> https://en.wikipedia.org/wiki/Oak_Park,_Illinois

### DATA REQUIREMENTS


For this study, we will leverage data from Python library Geocode & Foursquare to compare Evanston and Oak Park based on a variety of options including restaurants, entertainment, and public interest places. Here's how I will get & process the data for this notebook: 

1) From Geocode I will create a querry to get the latitude and longitude of both Evanston and Oak Park

2) I will then utilize the Foursquare API to fetch data on all the venues and point of interest within 1km of the latitude and longitude acquired in step # 1

3) I will process and clean-up the json file in order to convert to a dataframe and use with the pandas library

4) With the dataframe we will use one hot encoding and classify each venue type by frequencies for both neighborhoods. This should make comparison straighforward

5) Finally we will leverage clustering and other machine learning methods to further deepen our analysis and compare how similar or dissimilar both neighborhoods are

 